# Session 11: Transfer Learning

Let's see how to use transfer learning to improve our cat/dog model
and to do better image similarity metrics.

In [ ]:
%pylab inline

import numpy as np
import scipy as sp
import pandas as pd
import sklearn
from sklearn import linear_model
import urllib

import os
from os.path import join

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.rcParams["figure.figsize"] = (8,8)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.optimizers import SGD, RMSprop

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## Transfer learning

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.models import Model

We start by loading a particular neural network model called VGG19. It
contains 25 layers and over 143 million parameters. The code below reads
in the entire model and prints out it structure (unless keras is unavailable,
in which case a saved version of the model is printed just for reference).

In [ ]:
vgg19_full = VGG19(weights='imagenet')
vgg19_full.summary()

In [ ]:
img_path = join("..", "images", "test", "dog.jpg")
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
x.shape

In [ ]:
plt.imshow(img)
plt.axis("off")

In [ ]:
y = vgg19_full.predict(x)
for pred in decode_predictions(y)[0]:
    print(pred)

## Transfer learning

The VGG19 model was constructed in order to predict the objects present in an image,
but there is a lot more that we can do with the model. The amazing property of deep
learning is that the intermediate results in the neural network operate by detecting
lower-level features of the image. For example, the first few detect edges and textures,
the next few by understanding shapes, and the latter ones put these together to detect
objects. This is incredibly useful because it means that looking at the intermediate
outputs can tell us something interesting about the images beyond just the 1000
predicted categories.

Assuming the keras module is installed, we will create a new model that outputs the
second-to-last output of the model. The prediction of this contains 4096 dimensions.
These do not correspond directly to categories, but (in theory) images containing
similar objects should have similar 4096-dimensional values.

In [ ]:
vgg_fc2 = Model(inputs=vgg19_full.input, outputs=vgg19_full.get_layer('fc2').output)
vgg_fc2.summary()

And then, we will apply this to the dog image.

In [ ]:
embed = vgg_fc2.predict(x, verbose=True)
embed.shape

These numbers on their own do not really mean anything specific, but
they do serve as abstract features that we could use in a next 
processing step.

## Applying transfer learning to cats and dogs

Let us now apply the transfer learning algorithm to the cats and
dogs dataset. Note that this may take a while to run (around 10-15
minutes).

In [ ]:
df = pd.read_csv(join("..", "data", "catdog.csv"))
y = np.int32(df.animal.values == "dog")

In [ ]:
output = np.zeros((len(df), 224, 224, 3))

for i in range(len(df)):
    img_path = join("..", "images", "catdog", df.filename[i])
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    output[i, :, :, :] = x
    if (i % 100) == 0:
        print("Loaded image {0:03d}".format(i))

output = preprocess_input(output)
img_embed = vgg_fc2.predict(output, verbose=True)

Check the output of the model, and make sure you understand why this
image size makes sense.

In [ ]:
img_embed.shape

## Embedding model

We can use the embedding features to build a predictive model for the
dogs and cats. Let's see how well this works. Note that we no longer
need convolutions because the input is no longer raw image data.

In [ ]:
model = Sequential()
model.add(Dense(32, input_shape=(img_embed.shape[1],), activation="relu"))
model.add(Dense(32, activation="relu"))

model.add(Dense(units=2, activation="softmax"))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=True),
              metrics=['accuracy'])

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(img_embed, y)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32,
          validation_data=(X_test, y_test))

In [ ]:
yhat = model.predict_classes(X_test)
sklearn.metrics.accuracy_score(y_test, yhat)

We see that the predictive model is significantly better through the
use of transfer learning. And here are the few remaining mistakes:

In [ ]:
plt.figure(figsize=(14, 14))

yhat = model.predict_classes(img_embed)

idx = np.where(yhat != y)[0][:15]

for ind, i in enumerate(idx):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(5, 3, ind + 1)

    img = imread(join('..', 'images', 'catdog', df.filename[i]))
    plt.imshow(img)
    plt.axis("off")

## Recommendation system

Another thing that we can do with deep learning embeddings is to use
the features to find images that are close to one another.

In [ ]:
plt.figure(figsize=(14, 14))

ref_img_num = 750  # change this number!

print(df.iloc[ref_img_num])
idx = np.argsort(np.sum(np.abs(img_embed - img_embed[ref_img_num, :])**2, axis=1))[:9]

for ind, i in enumerate(idx):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(3, 3, ind + 1)

    img = imread(join('..', 'images', 'catdog', df.filename[i]))
    plt.imshow(img)
    plt.axis("off")

Try some different numbers and see what happens!